# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win percentage's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win percentage's correlation with the population of the city it is in for the **NHL** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [1]:
def nhl_correlation(): 

    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean up the NHL column
    cities.NHL=cities.NHL.str.split("[").str[0]
    # drop rows containing empty strings and dashes in the NHL column
    nhl=cities.drop(cities.loc[cities.NHL.isin(["", "—"])].index, axis=0)
    # create new rows for each of the NHL teams which were stuck together
    nhl=nhl.assign(NHL=nhl["NHL"].str.findall(r"[A-Z](?:\s[A-Z]|[^A-Z])+")).explode("NHL").reset_index(drop=True)
    # create a column containing the last word from the NHL column
    nhl["NHL_last"]=nhl.NHL.str.split(" ").str[-1]
    
    # subset for 2018 data
    nhl_df=nhl_df.loc[nhl_df.year==2018]
    # drop rows containing the word "Division" in the team column
    nhl_df.drop(nhl_df.loc[nhl_df.team.str.contains('Division')].index, axis=0, inplace=True)
    # clean up the team column
    nhl_df.team=nhl_df.team.str.replace("*", "")
    # create a column containing the last word from the team column
    nhl_df["NHL_last"]=nhl_df.team.str.split(" ").str[-1]

    # merge both dataframes on NHL_last column
    merged_nhl=pd.merge(nhl_df, nhl, how="outer", on="NHL_last")
    
    # convert data types of W and L columns from object to int
    merged_nhl["W"]=merged_nhl["W"].astype("int")
    merged_nhl["L"]=merged_nhl["L"].astype("int")
    
    # create a column with win-loss ratio calculated as W/(W+L)
    merged_nhl["WL_ratio"]=merged_nhl["W"].div((merged_nhl["W"].add(merged_nhl["L"])))
    
    # rename Population column name
    merged_nhl.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    
    # get the population and win-loss ratio of each metropolitan area averaged over those of multiple teams
    nhl_corr=merged_nhl.groupby(["Metropolitan area", "Population"]).agg({"WL_ratio":"mean"})

    # remove multi-indices
    nhl_corr.reset_index(inplace=True)
    
    # convert data type of Population column from object to int
    nhl_corr.Population=nhl_corr.Population.astype("int")
    
    population_by_region = list(nhl_corr.Population) # pass in metropolitan area population from cities
    win_loss_by_region = list(nhl_corr.WL_ratio) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return population_by_region, win_loss_by_region, correlation

# YOUR CODE HERE
# raise NotImplementedError()

In [2]:
population_by_region, win_loss_by_region, correlation = nhl_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"


## Question 2
For this question, calculate the win percentage's correlation with the population of the city it is in for the **NBA** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [3]:
def nba_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean up the NBA column
    cities.NBA=cities.NBA.str.split("[").str[0]
    # drop rows containing empty strings and dashes in the NBA column
    nba=cities.drop(cities.loc[cities.NBA.isin(["", "—"])].index, axis=0)
    # create new rows for each of the NBA teams which were stuck together
    nba=nba.assign(NBA=nba["NBA"].str.findall(r"[A-Z](?:\s[A-Z]|[^A-Z])+")).explode("NBA").reset_index(drop=True)
    # create a column containing the last word from the NBA column
    nba["NBA_last"]=nba.NBA.str.split(" ").str[-1]
    # fill in the missing value of Philadelphia's NBA_last column
    nba.loc[nba["Metropolitan area"]=="Philadelphia", "NBA_last"]="76ers"
    
    # subset for 2018 data
    nba_df=nba_df.loc[nba_df.year==2018]
    # clean up the team column
    nba_df.team=nba_df.team.str.replace("*", "")
    # clean up the team column further
    nba_df.team=nba_df.team.str.split("(").str[0]
    # remove empty spaces in the team column
    nba_df.team=nba_df.team.str.strip()
    # create a column containing the last word from the team column
    nba_df["NBA_last"]=nba_df.team.str.split(" ").str[-1]
    
    # merge both dataframes on NBA_last column
    merged_nba=pd.merge(nba_df, nba, how="outer", on="NBA_last")
    
    # convert data types of W and L columns from object to int
    merged_nba["W"]=merged_nba["W"].astype("int")
    merged_nba["L"]=merged_nba["L"].astype("int")
    
    # create a column with win-loss ratio calculated as W/(W+L)
    merged_nba["WL_ratio"]=merged_nba["W"].div((merged_nba["W"].add(merged_nba["L"])))
    
    # rename Population column name
    merged_nba.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    
    # get the population and win-loss ratio of each metropolitan area averaged over those of multiple teams
    nba_corr=merged_nba.groupby(["Metropolitan area", "Population"]).agg({"WL_ratio":"mean"})

    # remove multi-indices
    nba_corr.reset_index(inplace=True)
    
    # convert data type of Population column from object to int
    nba_corr.Population=nba_corr.Population.astype("int")
    
    population_by_region = list(nba_corr.Population) # pass in metropolitan area population from cities
    win_loss_by_region = list(nba_corr.WL_ratio) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return population_by_region, win_loss_by_region, correlation

# YOUR CODE HERE
# raise NotImplementedError()

In [4]:
population_by_region, win_loss_by_region, correlation = nba_correlation()
assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"


## Question 3
For this question, calculate the win percentage's correlation with the population of the city it is in for the **MLB** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [5]:
def mlb_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean up the MLB column
    cities.MLB=cities.MLB.str.split("[").str[0]
    # drop rows containing empty strings and dashes in the MLB column
    mlb=cities.drop(cities.loc[cities.MLB.isin(["", "—"])].index, axis=0)
    # create new rows for each of the MLB teams which were stuck together
    mlb=mlb.assign(MLB=mlb["MLB"].str.findall(r"[A-Z](?:\s[A-Z]|[^A-Z])+")).explode("MLB").reset_index(drop=True)
    # create a column containing the last word from the MLB column
    mlb["MLB_last"]=mlb.MLB.str.split(" ").str[-1]
    # include White Sox instead of Sox in MLB_last column to distinguish between Boston Red Sox and Chicago White Sox
    mlb.loc[mlb["MLB"]=="White Sox", "MLB_last"]="White Sox"
    
    # subset for 2018 data
    mlb_df=mlb_df.loc[mlb_df.year==2018]
    # create a column containing the last word from the team column
    mlb_df["MLB_last"]=mlb_df.team.str.split(" ").str[-1]
    # include White Sox instead of Sox in MLB_last column to distinguish between Boston Red Sox and Chicago White Sox
    mlb_df.loc[mlb_df["team"]=="Chicago White Sox", "MLB_last"]="White Sox"

    # merge both dataframes on MLB_last column
    merged_mlb=pd.merge(mlb_df, mlb, how="outer", on="MLB_last")
    
    # create a column with win-loss ratio calculated as W/(W+L)
    merged_mlb["WL_ratio"]=merged_mlb["W"].div((merged_mlb["W"].add(merged_mlb["L"])))
    
    # rename Population column name
    merged_mlb.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    
    # get the population and win-loss ratio of each metropolitan area averaged over those of multiple teams
    mlb_corr=merged_mlb.groupby(["Metropolitan area", "Population"]).agg({"WL_ratio":"mean"})

    # remove multi-indices
    mlb_corr.reset_index(inplace=True)
    
    # convert data type of Population column from object to int
    mlb_corr.Population=mlb_corr.Population.astype("int")
    
    population_by_region = list(mlb_corr.Population) # pass in metropolitan area population from cities
    win_loss_by_region = list(mlb_corr.WL_ratio) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return population_by_region, win_loss_by_region, correlation

# YOUR CODE HERE
# raise NotImplementedError()

In [6]:
population_by_region, win_loss_by_region, correlation = mlb_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"


## Question 4
For this question, calculate the win percentage's correlation with the population of the city it is in for the **NFL** using **2018** data. The win percentage ratio should be calculated using the following formula: **win/(win+loss)**. Then, return (a) population per region, (b) win percentage ratio per region, and (c) the correlation between these two. 

In [7]:
def nfl_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # clean up the NFL column
    cities.NFL=cities.NFL.str.split("[").str[0]
    # drop rows containing empty strings and dashes in the NFL column
    nfl=cities.drop(cities.loc[cities.NFL.isin(["", "—", "— "])].index, axis=0)
    # create new rows for each of the NFL teams which were stuck together
    nfl=nfl.assign(NFL=nfl["NFL"].str.findall(r"[0-[A-Z](?:\s[A-Z]|[^A-Z])+")).explode("NFL").reset_index(drop=True)
    # create a column containing the last word from the NFL column
    nfl["NFL_last"]=nfl.NFL.str.split(" ").str[-1]
    
    # subset for 2018 data
    nfl_df=nfl_df.loc[nfl_df.year==2018]
    # drop rows containing certain unnecessary words in the team column
    nfl_df.drop(nfl_df.loc[(nfl_df.team.str.contains('AFC East'))|(nfl_df.team.str.contains('NFC East'))|(nfl_df.team.str.contains('AFC North'))|(nfl_df.team.str.contains('NFC North'))|(nfl_df.team.str.contains('AFC South'))|(nfl_df.team.str.contains('NFC South'))|(nfl_df.team.str.contains('AFC West'))|(nfl_df.team.str.contains('NFC West'))].index, axis=0, inplace=True)
    # clean up the team column
    nfl_df.team=nfl_df.team.str.replace("*", "").str.replace("+", "")
    # create a column containing the last word from the team column
    nfl_df["NFL_last"]=nfl_df.team.str.split(" ").str[-1]

    # merge both dataframes on NFL_last column
    merged_nfl=pd.merge(nfl_df, nfl, how="outer", on="NFL_last")
    
    # convert data types of W and L columns from object to int
    merged_nfl["W"]=merged_nfl["W"].astype("int")
    merged_nfl["L"]=merged_nfl["L"].astype("int")
    
    # create a column with win-loss ratio calculated as W/(W+L)
    merged_nfl["WL_ratio"]=merged_nfl["W"].div((merged_nfl["W"].add(merged_nfl["L"])))
    
    # rename Population column name
    merged_nfl.rename(columns={"Population (2016 est.)[8]": "Population"}, inplace=True)
    
    # get the population and win-loss ratio of each metropolitan area averaged over those of multiple teams
    nfl_corr=merged_nfl.groupby(["Metropolitan area", "Population"]).agg({"WL_ratio":"mean"})

    # remove multi-indices
    nfl_corr.reset_index(inplace=True)
    
    # convert data type of Population column from object to int
    nfl_corr.Population=nfl_corr.Population.astype("int")
    
    population_by_region = list(nfl_corr.Population) # pass in metropolitan area population from cities
    win_loss_by_region = list(nfl_corr.WL_ratio) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    correlation = stats.pearsonr(population_by_region, win_loss_by_region)[0]

    return population_by_region, win_loss_by_region, correlation

# YOUR CODE HERE
# raise NotImplementedError()

In [8]:
population_by_region, win_loss_by_region, correlation = nfl_correlation()

assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [9]:
def sports_team_performance():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    import itertools

    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]    
    
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    # run previous functions to get populations and win-loss ratios for each of the sports leagues
    nfl_pop, nfl_wl, nfl_corr=nfl_correlation()
    nba_pop, nba_wl, nba_corr=nba_correlation()
    nhl_pop, nhl_wl, nhl_corr=nhl_correlation()
    mlb_pop, mlb_wl, mlb_corr=mlb_correlation()
    
    # get dataframes with population and win-loss ratio columns for each of the sports leagues
    nfl=pd.DataFrame(list(zip(nfl_pop, nfl_wl)), columns=["Population", "WL_ratio"])
    nba=pd.DataFrame(list(zip(nba_pop, nba_wl)), columns=["Population", "WL_ratio"])
    nhl=pd.DataFrame(list(zip(nhl_pop, nhl_wl)), columns=["Population", "WL_ratio"])
    mlb=pd.DataFrame(list(zip(mlb_pop, mlb_wl)), columns=["Population", "WL_ratio"])
    
    # create sports league dataframe and string lists
    sports_df=[nfl, nba, nhl, mlb]
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    
    # create pairs of combinations of the sports leagues
    pairs_df=list(itertools.combinations(sports_df, 2))
    pairs=list(itertools.combinations(sports, 2))
    
    # create an empty dictionary which would contain p-values
    p={}
    # loop through the pairs of sports league dataframes
    for i, j in zip(pairs_df, pairs):
        # merge the pairs of sports league dataframes to find common metropolitan areas (based on population count)
        merged_df=pd.merge(i[0], i[1], how="inner", on="Population")
        # run paired t-tests based on WL_ratio columns and store the p-values in a dictionary with tuples of sports league pairs as keys
        p[(j[0], j[1])]=stats.ttest_rel(merged_df["WL_ratio_x"], merged_df["WL_ratio_y"])[1]

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    
    # create an empty dataframe which would house the p-values
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    # loop through the sports league pairs which are tuple keys in the p-value dictionary
    for a, b in p.keys():
        # populate the empty p-values dataframe with p-values for the sports league pairs
        p_values.loc[a, b]=p[a, b]
        p_values.loc[b, a]=p[a, b]
    
    return p_values

In [10]:
p_values = sports_team_performance()

assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
